# Python作用域
  
什么是作用域？根据写程序的实践经验，我们知道，作用域是针对变量来说的。它就像一个盒子，规定了变量的作用范围和生命周期。



## 作用域的产生

就作用域而言，Python与C、Java有着很大的区别。在Java中作用域是根据`{}`来划分的。但在Python中，没有`{}`，那是不是根据缩进来划分呢？答案是**NO**。
Python并不是所有的语句块中都会产生作用域。只有当变量在**Module**(模块)、**Class**(类)、**def**(函数)中定义的时候，才会有作用域的概念。看下面代码：


In [1]:
def func():
    variable = 100
    print(variable)
print(variable)

NameError: name 'variable' is not defined

在作用域中定义的变量，一般只在作用域中有效。 需要注意的是：在`if-elif-else`、`for-else、while`、`try-except\try-finally`等关键字的语句块中并不会产成作用域。看下面的代码：

In [2]:
if True:
    variable = 100
    print (variable)
print ("******")
print (variable)

100
******
100


## LEGB

先看一段代码：

In [3]:
# example.py
from math import pi

PI = "global PI variable"

class A():
    
    def __init__(self, x):
        self.x = x
    
    def add(self, y):
        return self.x + y
    
    def get_global(self):
        if True:
            s = "can be accessiable"
        print(PI, s)
    def get_built_in(self):
        if True:
            s = "can be accessiable"
        print("built in varialbe", pi, s)
    

a = A(10)
print(a.add(5))
a.get_global()
a.get_built_in()


15
global PI variable can be accessiable
built in varialbe 3.141592653589793 can be accessiable


为了更好的描述和理解作用域间的查找规则，程序员将作用域进行了分类，简称为**LEGB**

- Local Scope(局部作用域)：在类、函数中定义的变量，比如：`get_global_pi`函数中的变量`s = "can be accessiable"`
- Enclosed Scope(闭包作用域)：嵌套函数中，外层函数被内层函数调用的变量，比如：下面代码中outer函数中定义的变量`pi = 'outer pi variable'`
- Global Scope(全局作用域)：在类、函数之外的，又在文件之中的变量，比如：全局变量`PI = "global PI variable"`
- Built-in Scope(内建作用域)：从别的模块中`import`的变量，比如：`from math import pi`中的变量`pi`

In [4]:
# Enclosed Scope

pi = 'global pi variable'
  
def outer(): 
    pi = 'outer pi variable'
    x = 5
    def inner(): 
        # pi = 'inner pi variable' 
        print(pi) 
    inner() 
    print(pi)

outer()
print(pi)

outer pi variable
outer pi variable
global pi variable


为什么要这样划分呢？我们可以从下面的例子中来亲身感受下作用域的规则。

### 局部作用域和全局作用域

In [5]:
x = 15

def foo():
    print(x)

foo()

15


在函数的局部作用域中，可以访问到全局作用域中的变量`x`。需要注意的是，函数`foo`里是去读`x`，为什么要强调这个呢，看下面这个函数就知道了。

In [6]:
y = 15

def modify1():
    y = 20
    print(y)

modify1()
print(y)

20
15


我们可以看到，在`modify1`函数中对`y`变量进行赋值，但是在全局作用域中打印`y`的值，发现全局变量`y`并没有被修改。这是为什么呢？带着这个疑问，我们再看看下面的`modify2`函数。

In [7]:
z = 15

def modify2():
    z += 20
    print(z)

modify2()
print(z)

UnboundLocalError: local variable 'z' referenced before assignment

为什么会报`UnboundLocalError`呢？为了更好的分析，我们可以借助`dis`模块，通过`dis.dis()`来反汇编编译过的Python代码对象、字符串包含的源代码，显示出一个人类可读的版本。

In [8]:
import dis

print("foo")
dis.dis(foo)
print("modify1")
dis.dis(modify1)
print("modify2")
dis.dis(modify2)

foo
  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_GLOBAL              1 (x)
              4 CALL_FUNCTION            1
              6 POP_TOP
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE
modify1
  4           0 LOAD_CONST               1 (20)
              2 STORE_FAST               0 (y)

  5           4 LOAD_GLOBAL              0 (print)
              6 LOAD_FAST                0 (y)
              8 CALL_FUNCTION            1
             10 POP_TOP
             12 LOAD_CONST               0 (None)
             14 RETURN_VALUE
modify2
  4           0 LOAD_FAST                0 (z)
              2 LOAD_CONST               1 (20)
              4 INPLACE_ADD
              6 STORE_FAST               0 (z)

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                0 (z)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
          

`modify2`函数报`UnboundLocalError`的根本原因是`LOAD_FAST`在`STORE_FAST`之前被调用了，也就是参数`z`在赋值之前被引用。那为什么CPython解释器知道使用`LOAD_FAST`而不是`LOAD_GLOBAL`? 也就是说为什么在编译阶段，CPython解释器就知道去就知道去哪个作用域找对应的变量了呢？

> One of the stages in the compilation of Python into bytecode is building the symbol table [[1]](https://eli.thegreenplace.net/2011/05/15/understanding-unboundlocalerror-in-python#id10). An important goal of building the symbol table is for Python to be able to mark the scope of variables it encounters - which variables are local to functions, which are global, which are free (lexically bound) and so on.

原来在将Python代码编译成字节码中，有一个阶段是构建**符号表**。构建符号表的一个重要目标是让Python能够标记它遇到的变量的范围，可以理解为，标记哪些变量是函数的局部变量，哪些是全局变量，哪些是自由变量(按词法绑定)，等等。

在生成字节码命令的时候，解释器遇到变量的时候，会去**符号表**查询该变量的标记，如果是标记是`Local`,那么加载该变量的指令就是`LOAD_FAST`；如果标记是非`Local`，那么就会去别的作用域找该变量，全局变量的加载指令就是`LOAD_GLOBAL`。

我们可以用`symtable`模块来查看`modify2`代码在编译成字节码的时候，添加到符号表里内容。

In [9]:
import symtable

code = '''
z = 15

def modify2():
    z += 20
    print(z)
'''

table = symtable.symtable(code, '<string>', 'exec')

foo_namespace = table.lookup('modify2').get_namespace()
sym_x = foo_namespace.lookup('z')

print(sym_x.get_name())
print(sym_x.is_local()) 

z
True


我们可以看到，`z`变量被标记为`modify2`函数的`Local`作用域。将变量标记为`Local`对于字节码的优化非常重要，因为编译器可以为它生成一条执行起来非常快的特殊指令。在这里我们只关注结果，就不深入地解释这个主题了。

### global关键字

In [10]:
z = 15

def modify_global():
    global z
    z = 20
    print(z)
modify_global()
print(z)

20
20


In [11]:
def modify_define_global():
    global var
    var = 34
    print(var)
modify_define_global()
print(var)

34
34


### nonlocal 关键字

`nonlocal`关键字可以用来绑定闭包作用域中的变量。

In [12]:
def outer(x):
    def inner_1(y):
        def inner_2(z):
            nonlocal x
            x = "inner_x"
            print("hello world! This is", z)
            print("free vars: x={}, y={}".format(x, y))
        inner_2("inner_2")
        print(y) 
    inner_1("inner_1")
    print(x)

outer("outer")

hello world! This is inner_2
free vars: x=inner_x, y=inner_1
inner_1
inner_x


但是`nonlocal`的关键字是在Python3中才存在的，Python2中没有。那在Python2中，如果需要对上层的值进行修改，就需要使用`dict`之类的对象绕过赋值语句的限制了。示例如下：

In [13]:
def outer():
    d = {"x": 1, "y": 2}
    l = [1,2]
    def inner():
        d["x"] = 3
        d["y"] = 4
        l.append(5)
        print(d,l)
    inner()
    print(d, l)
outer()
# dis.dis(outer)

{'x': 3, 'y': 4} [1, 2, 5]
{'x': 3, 'y': 4} [1, 2, 5]


## 闭包

对于嵌套函数，它可以访问到其外层作用域中声明的非局部（non-local）变量。比如下面示例代码中，变量`msg`可以被嵌套函数`printer`正常访问。

In [14]:
def print_msg():
    # print_msg 是外围函数
    msg = "zen of python"

    def printer():
        # printer是嵌套函数
        print(msg)
    printer()
# 输出 zen of python
print_msg()

zen of python


那么有没有一种可能即使脱离了函数本身的作用范围，局部变量还可以被访问得到呢？答案是闭包。

### 什么是闭包

函数身为第一类对象，它可以作为函数的返回值返回，现在我们来考虑如下的例子：

In [15]:
def print_msg():
    # print_msg 是外围函数
    msg = "zen of python"
    def printer():
        # printer 是嵌套函数
        print(msg)
    return printer

another = print_msg()
# 输出 zen of python
another()

zen of python


这段代码和前面例子的效果完全一样，同样输出`"zen of python"`。不同的地方在于内部函数`printer`直接作为返回值返回了。

一般情况下，函数中的局部变量仅在函数的执行期间可用，一旦`print_msg()`执行过后，我们会认为 msg变量将不再可用。然而，在这里我们发现 `print_msg`执行完之后，在调用`another`的时候`msg`变量的值正常输出了，这就是闭包的作用，闭包使得局部变量在函数外被访问成为可能。

看完这个例子，我们再来定义闭包，维基百科上的解释是:

> 在计算机科学中，闭包（Closure）是词法闭包（Lexical Closure）的简称，是引用了自由变量的函数。这个被引用的自由变量将和这个函数一同存在，即使已经离开了创造它的环境也不例外。所以，有另一种说法认为闭包是由函数和与其相关的引用环境组合而成的实体。

这里的`another`就是一个闭包，闭包本质上是一个函数，它有两部分组成，`printer`函数和变量`msg`。闭包使得这些变量的值始终保存在内存中。

闭包，顾名思义，就是一个封闭的包裹，里面包裹着自由变量，就像在类里面定义的属性值一样，自由变量的可见范围随同包裹，哪里可以访问到这个包裹，哪里就可以访问到这个自由变量。

所有函数都有一个`__closure__`属性，如果这个函数是一个闭包的话，那么它返回的是一个由`cell`对象组成的元组对象。`cell`对象`cell_contents`属性就是闭包中的自由变量。我们可以观察下

In [16]:
print(print_msg.__closure__)
print(another.__closure__)
print(another.__closure__[0].cell_contents)

None
(<cell at 0x10db9b110: str object at 0x10db9b370>,)
zen of python


### 闭包作用域

现在我们在回顾下闭包作用域。闭包作用域是针对这些和闭包函数绑定的自由变量来说的。

In [17]:
def outer(x):
    def inner_1(y):
        def inner_2(z):
            """inner doc
            """
            print("hello world! This is", z)
            print("free vars: x={}, y={}".format(x, y))
        print(y)
        return inner_2
    print(x)
    return inner_1

f1 = outer("outer")
f = f1("inner_1")
f("inner_2")

# f1.__closure__[0].cell_contents
f.__closure__[1].cell_contents
# f.__code__


outer
inner_1
hello world! This is inner_2
free vars: x=outer, y=inner_1


'inner_1'

### 闭包的应用

闭包避免了使用全局变量，此外，闭包允许将函数与其所操作的某些数据（环境）关连起来。这一点与面向对象编程是非常类似的，在面对象编程中，对象允许我们将某些数据（对象的属性）与一个或者多个方法相关联。

一般来说，当对象中只有一个方法时，这时使用闭包是更好的选择。来看一个例子：

In [18]:
def pow_n(n):
    def power(num):
        return num ** n
    return power

square = pow_n(2)
cube = pow_n(3)
print(square(5))
print(cube(5))

25
125


这比用类来实现更优雅，此外**装饰器**也是基于闭包的一中应用场景。

In [19]:
def log(func):
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper

@log
def now():
    print('2015-3-25')
    
now()

call now():
2015-3-25
